### Play with networktables to see how the API is organized  - 20220313 CJH

In [1]:
import time
import networktables
import pandas as pd

In [2]:
ntinst = networktables.NetworkTablesInstance.getDefault()

#### can give nt multiple servers to try

In [3]:
servers = ["127.0.0.1", "10.24.29.2"] #, "roboRIO-2429-FRC.local"]  # need to add the USB one here
ntinst.startClient(servers=servers)

#### if it connects you can get the connection info - may have to wait a few seconds though

In [4]:
connections = ntinst.getConnections()

In [5]:
print(f'ID:{connections[0].remote_id}\tIP:{connections[0].remote_ip}')

ID:Robot	IP:127.0.0.1


#### looks like all entries are given their own subtable

In [46]:
table = ntinst.getGlobalTable()  # also NetworkTables.getTable('/')
table

In [48]:
[t for t in dir(table) if '_' not in t]

['addEntryListener',
 'addSubTableListener',
 'basenameKey',
 'clearFlags',
 'clearPersistent',
 'containsKey',
 'containsSubTable',
 'delete',
 'getBoolean',
 'getBooleanArray',
 'getEntry',
 'getFlags',
 'getHierarchy',
 'getInstance',
 'getKeys',
 'getNumber',
 'getNumberArray',
 'getPath',
 'getRaw',
 'getString',
 'getStringArray',
 'getSubTable',
 'getSubTables',
 'getValue',
 'isPersistent',
 'loadEntries',
 'normalizeKey',
 'putBoolean',
 'putBooleanArray',
 'putNumber',
 'putNumberArray',
 'putRaw',
 'putString',
 'putStringArray',
 'putValue',
 'removeEntryListener',
 'removeTableListener',
 'saveEntries',
 'setDefaultBoolean',
 'setDefaultBooleanArray',
 'setDefaultNumber',
 'setDefaultNumberArray',
 'setDefaultRaw',
 'setDefaultString',
 'setDefaultStringArray',
 'setDefaultValue',
 'setFlags',
 'setPersistent']

In [7]:
subtables = table.getSubTables()
type(subtables)

list

In [8]:
len(subtables)

245

In [9]:
set(subtables)

{'FMSInfo', 'LiveWindow', 'Ramsete', 'Rev', 'SmartDashboard'}

In [10]:
s = table.getSubTable('SmartDashboard')

In [11]:
set(s.getSubTables())

{'',
 'Auto Rotate IMU',
 'AutoFetchBall',
 'AutoRotateSparkmax',
 'Field',
 'Tune Sparkmax',
 'auto lower group',
 'auto pickup',
 'auto shoot',
 'auto stage two',
 'autonomous routines',
 'autonomous_ramsete',
 'hold to feed',
 'intake motor',
 'intake piston',
 'path velocity',
 'ramsete path',
 'toggle compressor',
 'toggle shooter'}

In [12]:
unique = list(set(subtables))
frequency = {}
for item in unique:
    frequency[item] = subtables.count(item)
print("Frequency of items: ", frequency)

Frequency of items:  {'Ramsete': 8, 'Rev': 14, 'LiveWindow': 92, 'FMSInfo': 9, 'SmartDashboard': 122}


In [13]:
r = table.getSubTable('Ramsete')

#### get all entries - not sure if this is the fastest way to do it

In [14]:
entries = ntinst.getEntries('/', types=0)

In [15]:
sorted_tree = sorted([e.getName() for e in entries])
sorted_tree[::10]

['/FMSInfo/.type',
 '/LiveWindow/Ungrouped/.type',
 '/LiveWindow/Ungrouped/Compressor[0]/.name',
 '/LiveWindow/Ungrouped/Indexer/.default',
 '/LiveWindow/Ungrouped/Intake/.type',
 '/LiveWindow/Ungrouped/Pneumatics/.default',
 '/LiveWindow/Ungrouped/Shooter/.command',
 '/LiveWindow/Ungrouped/Solenoid[0,1]/.actuator',
 '/LiveWindow/Ungrouped/Solenoid[0,3]/.type',
 '/LiveWindow/Ungrouped/Vision/.name',
 '/LiveWindow/Ungrouped/navX-Sensor[4]/Value',
 '/Rev/kD_vel',
 '/Rev/pos_sp',
 '/SmartDashboard//Vision/ball/rotation',
 '/SmartDashboard/Auto Rotate IMU/running',
 '/SmartDashboard/Field/.name',
 '/SmartDashboard/auto lower group/.type',
 '/SmartDashboard/auto stage two/.controllable',
 '/SmartDashboard/autonomous routines/options',
 '/SmartDashboard/compressor_state',
 '/SmartDashboard/hub_distance',
 '/SmartDashboard/intake motor/running',
 '/SmartDashboard/path velocity/.controllable',
 '/SmartDashboard/ramsete path/.type',
 '/SmartDashboard/toggle compressor/running']

#### why do i have to take what was a table and get a flat list?

In [16]:
nt_dict = {}
for subtable in sorted(set(subtables)):
    child = [entry[len(subtable)+2:] for entry in sorted_tree if subtable in entry]
    nt_dict.update({subtable:child})
#nt_dict

#### data dump from networktables

In [24]:
for item in sorted_tree:
    entry = ntinst.getEntry(item)
    entry_val = entry.getValue()
    name = entry.getName()
    # value = val.getDouble() if val.isDouble() else val.getBoolean() if val.isBoolean else val.getString if val.isString else ''
    value = entry_val.value()
    print(f'Name: {entry.getName():60s} Value:{str(value):20s} \t Age: {int(time.time() - entry_val.time()/1E6):6d}  Type: {entry.getType()}  RPC:{entry_val.isRaw()} Levels:{name[1:].split("/")}')

Name: /FMSInfo/.type                                               Value:FMSInfo              	 Age:    459  Type: NetworkTableType.kString  RPC:False Levels:['FMSInfo', '.type']
Name: /FMSInfo/EventName                                           Value:                     	 Age:    459  Type: NetworkTableType.kString  RPC:False Levels:['FMSInfo', 'EventName']
Name: /FMSInfo/FMSControlData                                      Value:33.0                 	 Age:    459  Type: NetworkTableType.kDouble  RPC:False Levels:['FMSInfo', 'FMSControlData']
Name: /FMSInfo/GameSpecificMessage                                 Value:                     	 Age:    459  Type: NetworkTableType.kString  RPC:False Levels:['FMSInfo', 'GameSpecificMessage']
Name: /FMSInfo/IsRedAlliance                                       Value:True                 	 Age:    459  Type: NetworkTableType.kBoolean  RPC:False Levels:['FMSInfo', 'IsRedAlliance']
Name: /FMSInfo/MatchNumber                                         Va

In [18]:
ntdict = {}
for item in sorted_tree:
    entry = ntinst.getEntry(item)
    entry_val = entry.getValue()
    name = entry.getName()
    value = entry_val.value()
    ntdict.update({name:{'nt_path': entry.getName(), 'value':value, 'entry':entry, 'type':entry.getType()}})
    #print(f'Name: {entry.getName():60s} Value:{str(value):20s} \t Age: {int(time.time() - entry_val.time()/1E6):6d}  Type: {entry.getType()}  Levels:{name[1:].split("/")}')
df = pd.DataFrame(ntdict).transpose()
display(df)

,nt_path,value,entry,type
/FMSInfo/.type,/FMSInfo/.type,FMSInfo,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kString
/FMSInfo/EventName,/FMSInfo/EventName,,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kString
/FMSInfo/FMSControlData,/FMSInfo/FMSControlData,33.0,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kDouble
/FMSInfo/GameSpecificMessage,/FMSInfo/GameSpecificMessage,,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kString
/FMSInfo/IsRedAlliance,/FMSInfo/IsRedAlliance,True,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kBoolean
...,...,...,...,...
/SmartDashboard/toggle compressor/running,/SmartDashboard/toggle compressor/running,False,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kBoolean
/SmartDashboard/toggle shooter/.controllable,/SmartDashboard/toggle shooter/.controllable,True,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kBoolean
/SmartDashboard/toggle shooter/.name,/SmartDashboard/toggle shooter/.name,toggle shooter,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kString
/SmartDashboard/toggle shooter/.type,/SmartDashboard/toggle shooter/.type,Command,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kString


In [25]:
t = ntinst.getEntry('/SmartDashboard/autonomous_ramsete/running')

In [26]:
t

In [27]:
[item for item in dir(t) if '_' not in item ]

['Flags',
 'addListener',
 'clearFlags',
 'clearPersistent',
 'delete',
 'exists',
 'forceSetBoolean',
 'forceSetBooleanArray',
 'forceSetDouble',
 'forceSetDoubleArray',
 'forceSetRaw',
 'forceSetString',
 'forceSetStringArray',
 'forceSetValue',
 'getBoolean',
 'getBooleanArray',
 'getDouble',
 'getDoubleArray',
 'getFlags',
 'getHandle',
 'getInfo',
 'getInstance',
 'getLastChange',
 'getName',
 'getRaw',
 'getString',
 'getStringArray',
 'getType',
 'getValue',
 'isPersistent',
 'removeListener',
 'setBoolean',
 'setBooleanArray',
 'setDefaultBoolean',
 'setDefaultBooleanArray',
 'setDefaultDouble',
 'setDefaultDoubleArray',
 'setDefaultRaw',
 'setDefaultString',
 'setDefaultStringArray',
 'setDefaultValue',
 'setDouble',
 'setDoubleArray',
 'setFlags',
 'setPersistent',
 'setRaw',
 'setString',
 'setStringArray',
 'setValue',
 'value']

In [28]:
t.getName()

'/SmartDashboard/autonomous_ramsete/running'

In [44]:
t.setBoolean(True)

True

In [45]:
info = t.getInfo()

In [35]:
[item for item in dir(info) if '_' not in item ]

['entry', 'flags', 'name', 'type']

In [36]:
t.getType()

<NetworkTableType.kBoolean: 1>

In [37]:
t.getLastChange()

1647361381780923

In [38]:
val = t.getValue()

In [402]:
[item for item in dir(val) if '_' not in item ]

['getBoolean',
 'getBooleanArray',
 'getDouble',
 'getDoubleArray',
 'getFactoryByType',
 'getRaw',
 'getRpc',
 'getString',
 'getStringArray',
 'isBoolean',
 'isBooleanArray',
 'isDouble',
 'isDoubleArray',
 'isRaw',
 'isRpc',
 'isString',
 'isStringArray',
 'isValid',
 'makeBoolean',
 'makeBooleanArray',
 'makeDouble',
 'makeDoubleArray',
 'makeRaw',
 'makeRpc',
 'makeString',
 'makeStringArray',
 'makeValue',
 'time',
 'type',
 'value']

In [403]:
val.getDouble()

0.0

In [404]:
val.last_change()

1647357832007693

In [405]:
val.time()

1647357832007693

In [406]:
val.time??

Docstring:
time(self: _pyntcore._ntcore.Value) -> int

Get the creation time of the value.

:returns: The time, in the units returned by nt::Now().
Type:      method


In [407]:
val.value()

0.0

In [408]:
# generate the dictionary
d={}
levels = [s[1:].split('/') for s in sorted_tree]
for path in levels:
    current_level = d
    for part in path:
        if part not in current_level:
            current_level[part] = {}
        current_level = current_level[part]
#d

In [409]:
levels[100]

['LiveWindow', 'Ungrouped', 'navX-Sensor[4]', 'Value']

In [410]:
d['LiveWindow']['Ungrouped']['navX-Sensor[4]']['Value']

{}

In [411]:
for item in sorted_tree:
    value = ntinst.getEntry(item).getValue().value()
    levels = item[1:].split('/')
    if len(levels) == 2:
        d[levels[0]][levels[1]] = value
    elif len(levels) == 3:
        d[levels[0]][levels[1]][levels[2]] = value
    elif len(levels) == 4:
        d[levels[0]][levels[1]][levels[2]][levels[3]] = value

In [412]:
d

{'FMSInfo': {'.type': 'FMSInfo',
  'EventName': '',
  'FMSControlData': 33.0,
  'GameSpecificMessage': '',
  'IsRedAlliance': True,
  'MatchNumber': 0.0,
  'MatchType': 0.0,
  'ReplayNumber': 0.0,
  'StationNumber': 1.0},
 'LiveWindow': {'.status': {'LW Enabled': False},
  'Ungrouped': {'.type': 'LW Subsystem',
   'AnalogInput[0]': {'.name': 'AnalogInput[0]',
    '.type': 'Analog Input',
    'Value': 0.0},
   'Climber': {'.command': 'none',
    '.default': 'none',
    '.hasCommand': False,
    '.hasDefault': False,
    '.name': 'Climber',
    '.type': 'Subsystem'},
   'Compressor[0]': {'.name': 'Compressor[0]',
    '.type': 'Compressor',
    'Enabled': False,
    'Pressure switch': False},
   'DifferentialDrive[1]': {'.actuator': True,
    '.name': 'DifferentialDrive[1]',
    '.type': 'DifferentialDrive',
    'Left Motor Speed': 2.305845418907407e-05,
    'Right Motor Speed': 2.305845418907407e-05},
   'Indexer': {'.command': 'none',
    '.default': 'none',
    '.hasCommand': False,
  

In [413]:
def depth(d):
    if isinstance(d, dict):
        return 1 + (max(map(depth, d.values())) if d else 0)
    return 0

In [414]:
depth(d)

6

In [415]:
depth(d['SmartDashboard']['intake piston'])

1

In [416]:
widget_dict = {'qlabel_camera_view': {'widget':'self.qlabel_camera_view', 'table':''},
        'qlabel_climber_indicator':{'widget':'self.qlabel_climber_indicator', 'table':''},
        }

In [417]:
for key, value in widget_dict.items():
    print(key, value)

qlabel_camera_view {'widget': 'self.qlabel_camera_view', 'table': ''}
qlabel_climber_indicator {'widget': 'self.qlabel_climber_indicator', 'table': ''}
